In [4]:
import os
import codecs
import re
from time import sleep

In [5]:
# get google cloud
!pip install --upgrade google-cloud-translate
from google.cloud import translate

Requirement already up-to-date: google-cloud-translate in c:\users\user\anaconda3\lib\site-packages (1.3.3)


In [6]:
# get nltk to calculate bleu score
!pip install --upgrade nltk
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

Requirement already up-to-date: nltk in c:\users\user\anaconda3\lib\site-packages (3.4)


# Zulu

In [40]:
# load data
en_file = codecs.open('../clean/en_zu/enzu_parallel.test.en', 'r')
zu_file = codecs.open('../clean/en_zu/enzu_parallel.test.zu', 'r')

en_sents = [re.sub(' +', ' ', line.rstrip(' \n')) for line in en_file.readlines()]
zu_sents = [re.sub(' +', ' ', line.rstrip(' \n')) for line in zu_file.readlines()]

# translate
if not os.path.exists('./translated.zu'):
    trans_file = codecs.open('./translated.zu','w')
    
    # translate 20 sentences at a time
    translate_client = translate.Client()
    target = 'zu'
    for i in range(0, len(en_sents) // 20):
        translations = translate_client.translate(
            en_sents[i*20:(i+1)*20],
            target_language=target,
            source_language='en'
        )

        for translation in translations:
            trans_file.write(translation['translatedText'])
            trans_file.write('\n')

        print("Translated %d sentences" %((i+1)*20))
        sleep(1.5)
        
    trans_file.close()
    
# calculate bleu score
trans_file = codecs.open('./translated.zu', 'r')
trans_sents = [line.rstrip('\n').split(' ') for line in trans_file.readlines()]
num_trans = len(trans_sents)

zu_refs = [[re.sub(r'\s+([?.!,"])', r'\1', line).split(' ')] for line in zu_sents]

bleu_1 = corpus_bleu(zu_refs[:num_trans], trans_sents, weights=(1, 0, 0, 0))*100
bleu_4 = corpus_bleu(zu_refs[:num_trans], trans_sents)*100

print('BLEU1: %.3f' %bleu_1)
print('BLEU4: %.3f' %bleu_4)

trans_file.close()
en_file.close()
zu_file.close()

BLEU1: 27.090
BLEU4: 7.546


In [15]:
num_trans

2880

In [37]:
n = 65

print(en_sents[n])
print(zu_refs[n])
print(trans_sents[n])
print(sentence_bleu(zu_refs[n], trans_sents[n])*100)


CS .0171 Annual contract for the supply , delivery , installation , testing and commissioning of fibre network monitoring system for 12/24/36 month period Closing date: 18 May 2007
[['CS.0171', 'Inkontileka', 'yonyaka', 'yokuthengisela,', 'ukudiliva,', 'ukufaka,', 'ukuhlola', 'nokubeka', 'esimweni', 'sokusebenza', 'i-fibre', 'network', 'monitoring', 'system', 'izinyanga', 'ezingu', '12/24/36', 'Kuvalwa:', '18', 'Meyi', '2007']]
['CS', '.0171', 'Inkontileka', 'yonyaka', 'yokuhlinzeka,', 'ukulethwa,', 'ukufakwa,', 'ukuhlolwa', 'nokuthunyelwa', 'kohlelo', 'lokuqapha', 'kwenethiwekhi', 'ye-fibre', 'ngo', '12/24/36', 'inyanga', 'inyanga', 'yokuvalwa:', '18', 'Meyi', '2007']
2.6616657200018398e-76


In [107]:
zu_sents[0]

'Imibhalo yamathenda itholakala eThekwini Electricity , 1 Jelf Taylor Crescent , Durban ngo 08:00 kuya ku 15:00 , ngeMisombuluko kuya'

# Afrikaans

In [ ]:
# load data
en_file = codecs.open('../clean/en_af/enaf_parallel.test.en', 'r', 'utf-8')
af_file = codecs.open('../clean/en_af/enaf_parallel.test.af', 'r', 'utf-8')

en_sents = [re.sub(' +', ' ', line.rstrip(' \n')) for line in en_file.readlines()]
af_sents = [re.sub(' +', ' ', line.rstrip(' \n')) for line in af_file.readlines()]

# translate
if not os.path.exists('./translated.af'):
    trans_file = codecs.open('./translated.af','w', 'utf-8')
    
    # translate 20 sentences at a time
    translate_client = translate.Client()
    target = 'af'
    for i in range(0, len(en_sents) // 20):
        translations = translate_client.translate(
            en_sents[i*20:(i+1)*20],
            target_language=target,
            source_language='en'
        )

        for translation in translations:
            trans_file.write(translation['translatedText'])
            trans_file.write('\n')

        print("Translated %d sentences" %((i+1)*20))
        sleep(2)
        
    trans_file.close()
    
# calculate bleu score
trans_file = codecs.open('./translated.af', 'r')
trans_sents = [line.rstrip('\n').split(' ') for line in trans_file.readlines()]
num_trans = len(trans_sents)

af_refs = [[re.sub(r'\s+([?.!,"])', r'\1', line).split(' ')] for line in af_sents]

bleu_1 = corpus_bleu(af_refs[:num_trans], trans_sents, weights=(1, 0, 0, 0))*100
bleu_4 = corpus_bleu(af_refs[:num_trans], trans_sents)*100

print('BLEU1: %.3f' %bleu_1)
print('BLEU4: %.3f' %bleu_4)

trans_file.close()
en_file.close()
zu_file.close()

Translated 20 sentences
Translated 40 sentences
Translated 60 sentences
Translated 80 sentences
Translated 100 sentences
Translated 120 sentences
Translated 140 sentences
Translated 160 sentences
Translated 180 sentences
Translated 200 sentences
Translated 220 sentences
Translated 240 sentences
Translated 260 sentences
Translated 280 sentences
Translated 300 sentences
Translated 320 sentences
Translated 340 sentences
Translated 360 sentences
Translated 380 sentences
Translated 400 sentences
Translated 420 sentences
Translated 440 sentences
Translated 460 sentences
Translated 480 sentences
Translated 500 sentences
Translated 520 sentences
Translated 540 sentences
Translated 560 sentences
Translated 580 sentences
Translated 600 sentences
Translated 620 sentences
Translated 640 sentences
Translated 660 sentences
Translated 680 sentences
Translated 700 sentences
Translated 720 sentences
Translated 740 sentences
Translated 760 sentences
Translated 780 sentences
Translated 800 sentences
Tran

In [15]:
num_trans

2880

In [37]:
n = 65

print(en_sents[n])
print(af_refs[n])
print(trans_sents[n])
print(sentence_bleu(af_refs[n], trans_sents[n])*100)


CS .0171 Annual contract for the supply , delivery , installation , testing and commissioning of fibre network monitoring system for 12/24/36 month period Closing date: 18 May 2007
[['CS.0171', 'Inkontileka', 'yonyaka', 'yokuthengisela,', 'ukudiliva,', 'ukufaka,', 'ukuhlola', 'nokubeka', 'esimweni', 'sokusebenza', 'i-fibre', 'network', 'monitoring', 'system', 'izinyanga', 'ezingu', '12/24/36', 'Kuvalwa:', '18', 'Meyi', '2007']]
['CS', '.0171', 'Inkontileka', 'yonyaka', 'yokuhlinzeka,', 'ukulethwa,', 'ukufakwa,', 'ukuhlolwa', 'nokuthunyelwa', 'kohlelo', 'lokuqapha', 'kwenethiwekhi', 'ye-fibre', 'ngo', '12/24/36', 'inyanga', 'inyanga', 'yokuvalwa:', '18', 'Meyi', '2007']
2.6616657200018398e-76


In [107]:
zu_sents[0]

'Imibhalo yamathenda itholakala eThekwini Electricity , 1 Jelf Taylor Crescent , Durban ngo 08:00 kuya ku 15:00 , ngeMisombuluko kuya'